<a href="https://colab.research.google.com/github/Egoluback/aijic2021_transport/blob/yarik/model4track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Any, Tuple, Union
import pickle

In [50]:
path = "./"

train_labeled = pd.read_csv(path + 'data/base_files/labled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_labeled = pd.read_csv(path + 'data/base_files/labled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_labeled = pd.read_csv(path + 'data/labled_train_tracks_speed.csv', index_col=0, sep=',', comment='#')

train_unlabeled = pd.read_csv(path +'data/base_files/unlabled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_unlabeled = pd.read_csv(path+ 'data/base_files/unlabled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_unlabeled = pd.read_csv(path+ 'data/unlabled_train_tracks_speed.csv', index_col=0, sep=',', comment='#')

In [16]:
tracks_labeled

,Unnamed: 0.1,driver_id,dt,lat_,lon_,order_id,speed,is_aggressive
0,0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:43:41,55.792710,37.545409,001662da857b5a39bb402aacf3145f86,NaN,0.0
1,1,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:44:40,55.792013,37.544481,001662da857b5a39bb402aacf3145f86,5.906441,0.0
2,2,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:00,55.791365,37.543695,001662da857b5a39bb402aacf3145f86,15.696000,0.0
3,3,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:20,55.791267,37.543512,001662da857b5a39bb402aacf3145f86,2.862000,0.0
4,4,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:37,55.791175,37.543634,001662da857b5a39bb402aacf3145f86,2.710588,0.0
...,...,...,...,...,...,...,...,...
675217,675217,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:38:08,55.833583,37.490335,fffdfda358f35cf2d7f9d87d205655a7,36.891429,0.0
675219,675219,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:38:57,55.833093,37.491756,fffdfda358f35cf2d7f9d87d205655a7,8.517273,0.0
675220,675220,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:39:14,55.832650,37.493704,fffdfda358f35cf2d7f9d87d205655a7,27.783529,0.0
675222,675222,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:39:37,55.833031,37.495477,fffdfda358f35cf2d7f9d87d205655a7,23.720000,0.0


In [17]:
tracks_unlabeled.head()

""
",driver_id,dt,lat_,lon_,order_id,speed"
"0,6bcc649b6ec22251179da12125d04011,2021-03-29 11:55:56,55.7578864,37.40649060000001,0000a57c86cabd27d707a5fde1d0fbe4,"
"1,6bcc649b6ec22251179da12125d04011,2021-03-29 11:56:02,55.7578864,37.40649060000001,0000a57c86cabd27d707a5fde1d0fbe4,0.0"
"2,6bcc649b6ec22251179da12125d04011,2021-03-29 11:56:23,55.758017,37.40649970000001,0000a57c86cabd27d707a5fde1d0fbe4,2.228571428571428"
"3,6bcc649b6ec22251179da12125d04011,2021-03-29 11:56:43,55.7579873,37.4065001,0000a57c86cabd27d707a5fde1d0fbe4,0.54"
"4,6bcc649b6ec22251179da12125d04011,2021-03-29 11:57:04,55.7580145,37.4064947,0000a57c86cabd27d707a5fde1d0fbe4,0.5142857142857142"


each order is different batch

In [43]:
class Preprocessing_labeled():
    def __init__(self, tracks_labeled,  chunk_size=15, multiplier=1):
        self.tracks = tracks_labeled
        self.MULTIPLIER = multiplier
        self.CHUNK_SIZE = chunk_size

    # undersampling method deletes some extra non aggressive values
    def undersampling(self, X):
        aggressive_count = sum(X.is_aggressive == 1)
        non_aggressive_ind = X[X.is_aggressive == 0].index

        # keep aggressive_count*multiplier number of non_aggressive samples
        random_indices = np.random.choice(
            non_aggressive_ind, aggressive_count*self.MULTIPLIER, replace=False)
        return pd.concat([X.loc[random_indices], X[X.is_aggressive == 1]])

    def split(self, arr, chunk_size=15):
        result = []
        # get right length of arr so that it equally splits into chunks
        length = len(arr)
        split_length = length - (length % self.CHUNK_SIZE)

        for i in range(split_length)[self.CHUNK_SIZE::self.CHUNK_SIZE]:
            result.append(arr[i-self.CHUNK_SIZE:i])

        return np.array(result)

    # make df, so that each row has Series object with order speeds
    def make_nested(self, tracks, ):
        unique_orders = tracks.drop_duplicates('order_id', keep='last')
        unique_orders = self.undersampling(unique_orders)
        y_train = []
        X_train = []
        for order in unique_orders['order_id']:
            order_df = tracks[tracks.order_id == order]
            order_df.loc[0, 'speed'] = 0

            if order_df.shape[0] < self.CHUNK_SIZE:
                continue

            splitted_arrs = self.split(order_df.values)
            for arr in splitted_arrs:
                speed_series = []
                is_aggressive = arr[0][7]
                y_train.append(is_aggressive)

                for row in arr:
                   # append only speed
                    speed_series.append(row[6])
                X_train.append(pd.Series(speed_series))

        X_train = pd.DataFrame({'speed': X_train})
        y_train = np.array(y_train)
        return X_train, y_train


In [42]:

class Preprocessing_unlabeled():
    def __init__(self, tracks_unlabeled, chunk_size=15, multiplier=1):
        self.CHUNK_SIZE = chunk_size
        self.MULTIPLIER = multiplier
        self.tracks = tracks_unlabeled

    def split(self, arr,):
        result = []
        # get right length of arr so that it equally splits into chunks
        length = len(arr)
        split_length = length - (length % self.chunk_size)

        for i in range(split_length)[self.CHUNK_SIZE:self.chunk_size]:
            result.append(arr[i-self.CHUNK_SIZE:i])

        return np.array(result)

        # undersampling method deletes some extra non aggressive values
    def undersampling(self, X, y):
        aggressive_count = np.sum(y)
        non_aggressive_ind = np.argwhere(y == 0).flatten()
        aggressive_ind = np.argwhere(y == 1).flatten()

        # keep aggressive_count*multiplier number of non_aggressive samples
        random_indices = np.random.choice(
            non_aggressive_ind, aggressive_count*self.MULTIPLIER, replace=False)

        undersampling_X = np.concatenate(
            [np.take(X, random_indices), np.take(X, aggressive_ind)])
        undersampling_y = np.concatenate(
            [np.take([y, aggressive_ind]), np.take(y, random_indices)])
        return undersampling_X, undersampling_y

    def make_nested(self):
        unique_orders = self.tracks.drop_duplicates('order_id', keep='last')
        X_train = []
        for order in unique_orders['order_id']:
            order_df = self.tracks[self.tracks.order_id == order]
            order_df.loc[0, 'speed'] = 0

            if order_df.shape[0] < self.CHUNK_SIZE:
                continue

            splitted_arrs = self.split(order_df.values, self.CHUNK_SIZE)
            for arr in splitted_arrs:
                speed_series = []

                for row in arr:
                    # append only speed
                    speed_series.append(row[5])
                X_train.append(pd.Series(speed_series))

        X_train = pd.DataFrame({'speed': X_train})
        return X_train


In [47]:
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.rocket import MiniRocket
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
import sklearn.linear_model

#Classifier = Union[LogisticRegressionCV,
#                   LogisticRegressionCV, RidgeClassifier, RidgeClassifierCV]


class Model4track():
    def __init__(self, train_labeled, tracks_labeled, tracks_unlabeled, get_score=True):
        self.rocket = MiniRocket()
        self.get_score = get_score
        self.train_labeled = train_labeled
        self.tracks_unlabeled = tracks_unlabeled
        self.tracks_labeled = tracks_labeled
        self.preprocessing_unlabeled = Preprocessing_unlabeled(
            self.tracks_unlabeled,15,1)
        self.preprocessing_labeled = Preprocessing_labeled(
            self.tracks_labeled,15,1)

    def split_transform(self, X_train, y_train):
        X_train, y_train, X_test, y_test = train_test_split(
            X_train, y_train, test_size=0.33)
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        print("X_TRAIN IN SPLIT_TRANSFORM")
        print(X_train)
        self.rocket.fit(X_train)

        X_train_transform = self.rocket.transform(X_train)
        X_test_transform = self.rocket.transform(X_test)

        return X_train_transform, y_train, X_test_transform, y_test

    def train(self, X_train_transform, y_train, X_test_transform, y_test):

        classifier = RidgeClassifier(normalize=True)
        if self.get_score:
            classifier = RidgeClassifierCV(normalize=True)
        classifier.fit(X_train_transform, y_train)
        if self.get_score:
            classifier.score(X_test_transform, y_test)

        return classifier

    def pseudo_transformed_data(self,preprocessed_X,classifier):

        unlabeled_X_transform = self.rocket.transform(preprocessed_X)
        pseudo_y = classifier.predict(unlabeled_X_transform)

        return unlabeled_X_transform, pseudo_y

    def ss_train(self) -> sklearn.linear_model:
        # supervised learning
        X_train, y_train = self.preprocessing_labeled.make_nested(self.tracks_labeled)
        X_train_transform, y_train, X_test_transfrom, y_test = self.split_transform(
            X_train, y_train)
        classifier = self.train(
            X_train_transform, y_train, X_test_transfrom, y_test)

        with open('./models/classifier_labeled_trained.pkl', 'wb') as file:
            pickle.dump(classifier, file)

        # unsupervised learning
        unlabeled_X = self.preprocessing_unlabeled.make_nested(self.tracks_unlabeled)
        X_pseudo_transform, pseudo_y = self.pseudo_transformed_data(
            unlabeled_X, classifier)
        # undersampling_unlabeled(unlabeled_X_transform, pseudo_y, multiplier=1)
        all_X_transform = pd.concat([X_pseudo_transform, X_train_transform])
        all_y = pd.concat(pseudo_y, y_train)
        print('LENGTH')
        print(all_X)
        print('AGGRESSIVE PERCENTAGE')
        print(len(all_y)/all_y.sum())


        # Rocket developers recommend use logistic regression train using stochastic gradient descent for datasets larger 20k examples
        pseudo_classifer = self.train(all_X_transform, all_y, X_test_transfrom, y_test, get_score=True) 
        with open('./models/classifier_pseudo_trained.pkl', 'wb') as file:
            pickle.dump(pseudo_classifer, file)




Self-training realization:
1. Make pseudo-labels for unlabled data
2. Retrain classifier on labled and pseudolabled data
https://towardsdatascience.com/a-gentle-introduction-to-self-training-and-semi-supervised-learning-ceee73178b38 

In [49]:
model = Model4track(train_labeled, tracks_labeled, tracks_unlabeled)
model.ss_train()

<ipython-input-43-5ca8bd67bf4f>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_df.loc[0, 'speed'] = 0


X_TRAIN IN SPLIT_TRANSFORM
                                                  speed
732   0      0.041499
1      3.702857
2      0.84521...
773   0     44.262000
1     24.399101
2     33.31317...
8     0     -0.049510
1     21.265000
2     18.97411...
605   0     46.881818
1     39.210000
2     20.86434...
592   0     45.993600
1     47.253913
2     50.80500...
...                                                 ...
323   0     -0.001540
1      1.367395
2     21.30857...
967   0     39.534545
1     86.089091
2     84.73090...
470   0     20.192727
1     28.230698
2     58.76571...
1272  0      2.304000
1      4.320000
2     12.50526...
1611  0     -0.026508
1      7.416000
2      4.03363...

[1139 rows x 1 columns]


ValueError: If passed as a np.array, X must be a 3-dimensional array, but found shape: (1139,)

In [43]:
all_data_transformed = pd.concat([X_train_transform, X_pseudo_transformed])
